In [1]:
from lightfm import LightFM
from lightfm.data import Dataset
from lightfm.cross_validation import random_train_test_split

from sklearn.neighbors import NearestNeighbors

import numpy as np
import pandas as pd

import itertools

/opt/anaconda3/lib/python3.7/site-packages/lightfm/_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn('LightFM was compiled without OpenMP support. '


In [2]:
interactions = pd.read_csv('data/interactions.csv')
interactions = interactions.rename(columns={'row': 'customer_id', 'col': 'item_id', 'data': 'response'})
interactions.columns = [x.lower() for x in interactions.columns]
#print('interactions')
#print(interactions.head())

#goods
item_asset = pd.read_csv('data/item_asset.csv').drop(['col'], axis=1)
item_asset = item_asset.rename(columns={'row': 'item_id', 'data': 'property_item'})
item_asset.columns = [x.lower() for x in item_asset.columns]
#print('item_asset')
#print(item_asset.head())

item_price = pd.read_csv('data/item_price.csv').drop(['col'], axis=1)
item_price = item_price.rename(columns={'row': 'item_id', 'data': 'price_scaled'})
item_price.columns = [x.lower() for x in item_price.columns]
#print('item_price')
#print(item_price.head())

item_subclass = pd.read_csv('data/item_subclass.csv')
item_subclass = item_subclass.rename(columns={'row': 'item_id', 'col': 'category_num', 'data': 'category_connection'})
item_subclass.columns = [x.lower() for x in item_subclass.columns]
#print('item_subclass')
#print(item_subclass.head())

#users
user_age = pd.read_csv('data/user_age.csv').drop(['col'], axis=1)
user_age = user_age.rename(columns={'row': 'customer_id', 'data': 'age_scaled'})
user_age.columns = [x.lower() for x in user_age.columns]
#print('user_age')
#print(user_age.head())

user_region = pd.read_csv('data/user_region.csv')
user_region = user_region.rename(columns={'row': 'customer_id', 'col': 'region_ohe_position', 'data': 'region_feat'})
user_region.columns = [x.lower() for x in user_region.columns]
#print('user_region')
#print(user_region.head())


In [3]:
print(f'interactions shape: {interactions.shape}')
interactions.head()

interactions shape: (398636, 3)


customer_id  item_id  response
0            0     3568       1.0
1            0     3827       1.0
2            0     4844       1.0
3            0     5734       1.0
4            0     6518       1.0

In [4]:
interactions['customer_id'].unique().shape, interactions['item_id'].unique().shape

((27255,), (15277,))

In [5]:
interactions_matrix = interactions.pivot(index='customer_id', columns='item_id', values='response').fillna(0)

In [6]:
flag = False
for i, x in enumerate(interactions_matrix.values.sum(axis=1)):
    if not x:
        flag = True
        print(i)
if not flag:
    print('all users have at least 1 connection')

all users have at least 1 connection


In [5]:
#group tables for more convenient work
print(f'item_asset shape: {item_asset.shape}, item_price shape: {item_price.shape}, \
item_subclass shape: {item_subclass.shape}')

items = pd.merge(item_asset, item_price, how='inner', on='item_id')
items = pd.merge(items, item_subclass, how='inner', on='item_id')
print(f'items shape: {items.shape}')
items.head()

item_asset shape: (18490, 2), item_price shape: (18493, 2), item_subclass shape: (18495, 3)
items shape: (18488, 5)


item_id  property_item  price_scaled  category_num  category_connection
0        0       0.009497      0.012911           679                  1.0
1        1       0.004226      0.005211          1376                  1.0
2        2       0.003371      0.004131          1495                  1.0
3        3       0.002991      0.003521           502                  1.0
4        4       0.002991      0.003521           502                  1.0

In [6]:
items.isna().sum()

item_id                0
property_item          0
price_scaled           0
category_num           0
category_connection    0
dtype: int64

In [7]:
print(f'user_age shape: {user_age.shape}, user_region shape: {user_region.shape}')

users = pd.merge(user_age, user_region, how='inner', on='customer_id')
print(f'users shape: {users.shape}')
users.head()

user_age shape: (30317, 2), user_region shape: (26609, 3)
users shape: (25781, 4)


customer_id  age_scaled  region_ohe_position  region_feat
0            2         1.0                    5          1.0
1            7         1.0                    6          1.0
2           10         1.0                    7          1.0
3           12         1.0                    7          1.0
4           13         1.0                    7          1.0

In [8]:
users.isna().sum()

customer_id            0
age_scaled             0
region_ohe_position    0
region_feat            0
dtype: int64

**Little preprocessing**

In [ ]:
users_interact_id = set(interactions['customer_id'].unique())
users_info_id = set(users['customer_id'].unique())
print(len(users_info_id - users_interact_id), len(users_interact_id - users_info_id))

items_interact_id = set(interactions['item_id'].unique())
items_info_id = set(items['item_id'].unique())
print(len(items_info_id - items_interact_id), len(items_interact_id - items_info_id))

Here's cold users with features and warm users without features.
<br>
Let's use ANN to find k nearest neighbors and take their features (if k=1 then it's clear, else if k > 1 then we can aggregate feature's values in some ways)

The same is for items

We can see that there're items_id and users_id without features so we can for example use KNN to find similar items_id and users_id and take features from there.
<br>
Also there're items_id and users_id without connections (aka new items_id/users_id) but with features, this is cold start problem but it seems that it can be solved based on their features information 

But know for simplicity let's take intersections of items_id and users_id

In [14]:
print(f'users shape: {users.shape}, items shape: {items.shape}, interactions shape: {interactions.shape}')

users shape: (25781, 4), items shape: (18488, 5), interactions shape: (398636, 3)


In [15]:
#так как были отброшены некотоыре покупатели, то могла пропасть и часть item которые брали только они
#так и произошло поэтому "просеим" items_id еще раз
interactions = interactions[(interactions['customer_id'].isin(users_id))]
items_id &= set(interactions['item_id'].unique())
interactions[(interactions['item_id'].isin(items_id))]

customer_id  item_id  response
13                2      736       1.0
14                2     1425       1.0
15                2     3653       1.0
16                2     4634       1.0
17                2     4653       1.0
...             ...      ...       ...
398375        30909      207       1.0
398376        30909     4482       1.0
398377        30909    15139       1.0
398378        30909    17367       1.0
398379        30909    18193       1.0

[340090 rows x 3 columns]

In [16]:
users = users[users['customer_id'].isin(users_id)]
items = items[items['item_id'].isin(items_id)]

In [17]:
print(f'users shape: {users.shape}, items shape: {items.shape}, interactions shape: {interactions.shape}')

users shape: (22754, 4), items shape: (14875, 5), interactions shape: (340130, 3)


**Simple lightfm**

In [18]:
def build_features(df, features_names):
    
    features = []
    columns = list(itertools.chain(*[[col]*len(df[col].unique()) for col in features_names]))
    unique_features = list(itertools.chain(*[list(df[col].unique()) for col in features_names]))
    
    for x, y in zip(columns, unique_features):
        features.append(f'{x}:{float(y)}')
    
    return features

In [19]:
print(users.columns)
print(items.columns)

Index(['customer_id', 'age_scaled', 'region_ohe_position', 'region_feat'], dtype='object')
Index(['item_id', 'property_item', 'price_scaled', 'category_num',
       'category_connection'],
      dtype='object')


In [20]:
user_features = build_features(users, users.columns[1:])
item_features = build_features(items, items.columns[1:])

In [21]:
print('user:', user_features[:3])
print('item:', item_features[:3])

user: ['age_scaled:1.0', 'region_ohe_position:5.0', 'region_ohe_position:6.0']
item: ['property_item:0.00949667616334283', 'property_item:0.003371320037986705', 'property_item:0.002991452991452991']


In [24]:
dataset = Dataset()

In [25]:
#проверь тут!!!! скорей всего надо юзерс и айтем поправить
dataset.fit(users=users['customer_id'].unique(),
                    items=items['item_id'].unique(),
                    user_features=user_features,
                    item_features=item_features
                   )

In [26]:
interactions_matrix, weights = dataset.build_interactions([(x[0], x[1])for x in interactions.to_numpy()])

ValueError: Item id 15387.0 not in item id mapping. Make sure you call the fit method.

In [26]:
# Helper function that takes the user features and converts them into the proper "feature:value" format
def feature_colon_value(my_list, features_names):
    """
    Takes as input a list and prepends the columns names to respective values in the list.
    For example: if my_list = [1,1,0,'del'],
    resultant output = ['f1:1', 'f2:1', 'f3:0', 'loc:del']
   
    """
    result = []
    
    for x, y in zip(features_names, my_list):
        result.append(f'{x}:{float(y)}')
    
    return result

In [27]:
def build_features_list(df, features_names):
    
    ad_subset = df[features_names]
    ad_list = [list(x) for x in ad_subset.to_numpy()]
    features_list = []
    for val in ad_list:
        features_list.append(feature_colon_value(val, features_names))
        
    return features_list

In [28]:
user_features_list = build_features_list(users, users.columns[1:])
item_features_list = build_features_list(items, items.columns[1:])

In [29]:
user_tuple = list(zip(users['customer_id'].to_numpy(), user_features_list))
item_tuple = list(zip(items['item_id'].to_numpy(), item_features_list))

In [30]:
items[items['item_id']==103]

item_id  property_item  price_scaled  category_num  category_connection
103      103       0.003039      0.004225           855                  1.0

In [31]:
for x, y in item_tuple:
    if x==103:
        print(x)
        break

103


In [32]:
user_features = dataset.build_user_features(user_tuple, normalize=False)
item_features = dataset.build_item_features(item_tuple, normalize=False)

ValueError: item id 103 not in item id mappings.